[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/tp/01_aprendizaje_supervisado/tp01-enunciado.ipynb)

# Trabajo Práctico -  Aprendizaje supervisado
### Clasificación de expresiones genómicas

<span style="color: red;">**Fecha de entrega: Jueves 01 de mayo del 2025 - hasta las 17:00hs.**

<span style="color: red;">**Fecha de entrega intermedia: Jueves 17 de Abril del 2025 - hasta las 17:00hs.**
</span>

## Introducción

En el mundo actual, distintas disciplinas científicas empiezan, cada vez más, a interactuar con el fin de potenciar sus descubrimientos. En este caso dos grupos de investigación de [CONICET](https://www.conicet.gov.ar/) se embarcan en la combinación entre biología y informática para abordar la detección temprana y el pronóstico preciso de enfermedades como el cáncer. Este proyecto combina las tecnologías de secuenciación de nueva generación ([_NGS_](https://es.wikipedia.org/wiki/Secuenciaci%C3%B3n_paralela_masiva), por sus siglas en inglés) con la potencia de la inteligencia artificial. El enfoque se centra en un dataset único que abarca mediciones de [_ARN_](https://es.wikipedia.org/wiki/ARN_mensajero) de 200 [_genes_](https://es.wikipedia.org/wiki/Gen), recopiladas de pacientes con lesiones [_pre-tumorales_](https://en.wikipedia.org/wiki/Hyperplasia). Este conjunto de datos se convierte en una valiosa fuente de información para entender cómo las células en estado de hiperplasia pueden evolucionar hacia [_tumores malignos_](https://en.wikipedia.org/wiki/Neoplasm), una transformación que ha desconcertado a la ciencia durante décadas.

La hiperplasia, es un fenómeno en el que las células experimentan un crecimiento anormal y descontrolado, es un punto de partida crucial en nuestro análisis. ¿Cómo y por qué algunas células que experimentan hiperplasia se convierten en células cancerosas, mientras que otras no? Esta pregunta es el corazón de nuestra investigación. Para responderla se realizo un estudio donde se obtuvieron muestras de distintos tipos de hiperplasias de pacientes con antecedentes familiares y lesiones pre tumorales. Este grupo de pacientes, o cohorte, fue monitoreado periodicamente durante los siguientes 5 años buscando indicios de neoplasias o nuevas hiperplasias más agresivas. Con las muestras obtenidas en este estudio se realizo un [_biobanco_](https://en.wikipedia.org/wiki/Biobank) con las mediciones que habitualmente se hacen en la construccion de este tipo de [_plataformas_](https://xena.ucsc.edu/). Cada muestra fue etiquetada como **_buen pronostico_**, si no hubo indicios de nuevas hiperplasias o similares; contrariamente se etiquetaron como de **_mal pronostico_** si hubo una recaida.

Este trabajo se concentra en un panel de genes, especificamente en la expresion de 200 genes que se creen tienen un papel crucial en la transformacion tumoral y su etiqueta correspondiente.

En concreto:

Tendrán un archivo `.csv` en donde se almacenan:
  - una matriz de datos `X` de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$.
  - una columna llamada `target` que representa un vector de $500$ posiciones con dos posibles valores: `True` (ó 1, es decir, tiene buen pronostico) y `False` (ó 0, tiene mal pronostico).

Los datos están en esta [carpeta](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

Por otra parte, tendrán disponibles un conjunto de instancias sin etiquetas, que utilizaremos para comprobar la calidad de sus resultados (ver Ejercicio 5).

**Recomendamos fuertemente leer primero todo el enunciado del trabajo antes de empezar a trabajar sobre el problema propuesto.**

---

### Sobre el informe

Para este trabajo deberán entregar, además del código de las pruebas y experimentos que realicen, un informe en el que deberan seleccionar, para cada apartado, sus resultados acompañado de un texto que explique, reflexione, justifique y conluya dicho contenido.

Cada ejercicio indica el largo máximo del texto que se puede incluir. Los gráficos no están contados en dicho espacio.
Cada gráfico incluido debe contar con:
  
  - nombres de los ejes,
  - título,
  - leyenda autocontenida,
  - debe ser referenciado desde el texto, ya que su inclusión se da porque aporta a la discusión del trabajo.

**El informe no puede superar un máximo de 8 carillas (contando gráficos) o 4 hojas más carátula.** Tamaño de la letra: estandár de latex (10pt). No se corregirán trabajos que no cumplan con esta consigna.

---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Ejercicio 1

### Separación de datos

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos.

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. ¿Qué consideraciones tuvieron en cuenta para realizar esta división?

**Importante**: en este punto no está permitido dividir la base de datos utilizando la función `train_test_split` de sklearn. Deben decidir e implementar la separación.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/Mauro1642/Aprendizaje-Automatico/refs/heads/main/data.csv")
df

,QqgU,IJsJ,YlJA,tFLR,MYkJ,Cado,cmdU,AUYy,mVEY,zSZq,...,cgNV,naPw,EOre,sMSU,TihR,sbWq,aWBQ,fGxK,JWOq,target
0,-0.753155,-0.304399,0.307146,-1.377240,-1.975242,-0.052628,0.430082,-0.659100,4.897377,0.754531,...,0.975926,-0.593156,0.315798,0.546475,1.128513,7.654107,0.563439,0.322854,-1.754513,0
1,0.660977,-1.374880,-0.039899,-0.157981,-0.263226,-0.035127,1.583745,-0.143602,12.945694,-0.347768,...,-0.274560,0.205678,-0.296301,-1.004013,7.090070,0.182824,0.113722,1.323722,0.710734,0
2,1.906825,-0.308091,1.115546,0.093203,-1.384119,0.616543,0.384448,-0.817921,-7.673495,1.028706,...,-0.369924,-0.678204,1.564510,0.314275,-4.246162,-1.941577,-0.566316,-0.261689,-1.715006,1
3,3.767180,-0.495569,-0.654660,0.101866,0.644159,1.978273,-1.087526,0.925767,10.392570,0.203025,...,0.252199,-0.200540,1.043750,-0.039129,-2.898566,-5.231800,-0.476618,-0.300148,0.315923,0
4,3.540548,0.544672,-1.820594,0.068094,0.844530,-0.516157,0.561619,1.634488,1.220771,-0.344691,...,-2.380954,1.532346,-1.594870,-1.809599,-0.445860,-0.169324,1.876894,-0.043769,1.529304,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-6.196035,-0.627457,-0.803145,0.256876,0.592146,1.088430,-0.831851,-0.889525,-11.436139,0.542209,...,1.038811,-0.773806,-0.905051,-0.404457,0.871845,2.107610,-0.572656,-0.607867,1.170686,1
496,-1.045521,-0.875682,-0.829544,-1.494002,0.436961,-0.494770,-0.219712,-0.387546,-0.751816,0.051520,...,-1.157861,-2.462821,-0.571710,-0.239654,4.685438,1.120013,-0.723162,-0.155328,1.200592,0
497,-3.385825,-1.500261,-0.923048,0.002346,-1.189872,-1.471672,0.678777,-1.268629,-13.890491,1.880261,...,0.880166,1.157510,-0.135622,-0.866339,-3.999636,5.705475,0.426428,0.007213,1.047125,0
498,-5.068169,-0.783885,-0.819872,-0.906944,0.210135,0.629747,0.802087,0.333887,2.750179,-0.200211,...,-0.548944,0.258575,-0.522897,-0.139874,0.307522,2.683021,-0.336143,0.031017,-0.294661,0


In [4]:
# Veamos la proporción de negativos (0) y postivos (1):
df['target'].value_counts(normalize=True)

,proportion
target,
0,0.698
1,0.302


In [8]:
def stratified_split(df: pd.DataFrame, eval_prop: float) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Separa el dataset df en desarrollo y evaluación manteniendo en ambos la
    misma proporción de cada clase respecto al original.
    """
    n_samples = len(df)
    n_negativos, n_positivos = df["target"].value_counts()

    dev_prop = 1 - eval_prop

    # Mezclamos las instancias del dataset
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Separamos el dataframe de los negativos y positivos
    df_negativos = df[df["target"] == 0]
    df_positivos = df[df["target"] == 1]

    # Juntamos los primeros n_pos(neg)*dev_prop de c/u y los dejamos para desarrollo
    df_dev = pd.concat([
        df_positivos.iloc[:int(n_positivos*dev_prop)],
        df_negativos.iloc[:int(n_negativos*dev_prop)]
    ])
    # Los restantes quedan para evaluación
    df_eval = pd.concat([
        df_positivos.iloc[int(n_positivos*dev_prop):],
        df_negativos.iloc[int(n_negativos*dev_prop):]
    ])

    return df_dev, df_eval

In [9]:
# Dividimos el dataset en desarrollo y evaluación
df_dev, df_eval = stratified_split(df, eval_prop=0.2)
# Verificamos que la cantidad de instancias en c/u es la correcta es la correcta
assert len(df_dev) + len(df_eval) == len(df)

In [12]:
X_dev, y_dev = df_dev.drop(columns=["target"], ), df_dev["target"]
X_eval, y_eval = df_eval.drop(columns=["target"]), df_eval["target"]

In [13]:
print(f"Proporciones en desarrollo: {y_dev.value_counts(normalize=True)}")
print(f"Proporciones en evaluación: {y_eval.value_counts(normalize=True)}")

Proporciones en desarrollo: target
0    0.699248
1    0.300752
Name: proportion, dtype: float64
Proporciones en evaluación: target
0    0.693069
1    0.306931
Name: proportion, dtype: float64


## Ejercicio 2


### Construcción de modelos

Para este punto, la tarea consiste en construir y evaluar modelos de tipo **árbol de decisión**. Además, obtener una **estimación realista de la performance** de los mismos.

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

2. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_, y _Area Under the Receiver Operating Characteristic Curve (AUCROC)_.

   En esta oportunidad se va a pedir además de calcular las métricas para cada fold por separado y su promedio, que hagan el cálculo del score global (como vimos en clase), sólo para los folds de validación.

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score

# Metricas de evaluación
metricas = {
    "Accuracy": "accuracy",
    "AUPRC" : make_scorer(average_precision_score, response_method="predict_proba"),
    "AUCROC": make_scorer(roc_auc_score, response_method="predict_proba")
}

arbol = DecisionTreeClassifier(max_depth=3)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

scores = cross_validate(arbol, X_dev, y_dev,
                        cv=skf,
                        scoring=metricas,
                        return_train_score=True)

for key, value in scores.items():
  print(f"{key}: {value}, mean = {float(value.mean())}")
  print()

fit_time: [0.03070593 0.02914143 0.03076434 0.03217411 0.03018141], mean = 0.030593442916870116

score_time: [0.00864267 0.00864005 0.01525927 0.00813079 0.0082593 ], mean = 0.009786415100097656

test_Accuracy: [0.7        0.675      0.6375     0.6875     0.63291139], mean = 0.6665822784810127

train_Accuracy: [0.79623824 0.81191223 0.82131661 0.8369906  0.828125  ], mean = 0.8189165360501567

test_AUPRC: [0.41953255 0.42612482 0.32948336 0.44222435 0.32793947], mean = 0.3890609107391438

train_AUPRC: [0.65575862 0.67116959 0.71411069 0.7003898  0.71019614], mean = 0.6903249675583958

test_AUCROC: [0.66443452 0.67001488 0.578125   0.57626488 0.51931818], mean = 0.6016314935064935

train_AUCROC: [0.82677037 0.85299888 0.83742059 0.81266349 0.83656529], mean = 0.8332837232076127



In [45]:
# Calculamos las metricas globales global
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

arbol.fit(X_dev, y_dev)
y_pred = arbol.predict(X_eval)

print(f"Accuracy: {accuracy_score(y_eval, y_pred)}")
print(f"AUPRC: {average_precision_score(y_eval, y_pred)}")
print(f"AUCROC: {roc_auc_score(y_eval, y_pred)}")

Accuracy: 0.693069306930693
AUPRC: 0.3629830725007985
AUCROC: 0.5808755760368663


<table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUPRC (training)</th>
      <th>AUPRC (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td align="center">0.840</td>
      <td align="center">0.713</td>
      <td align="center">0.698</td>
      <td align="center">0.402</td>
      <td align="center">0.819</td>
      <td align="center">0.615</td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td align="center">0.809</td>
      <td align="center">0.663</td>
      <td align="center">0.656</td>
      <td align="center">0.362</td>
      <td align="center">0.842</td>
      <td align="center">0.585</td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td align="center">0.787</td>
      <td align="center">0.713</td>
      <td align="center">0.607</td>
      <td align="center">0.417</td>
      <td align="center">0.832</td>
      <td align="center">0.626</td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td align="center">0.843</td>
      <td align="center">0.625</td>
      <td align="center">0.680</td>
      <td align="center">0.344</td>
      <td align="center">0.813</td>
      <td align="center">0.591</td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td align="center">0.822</td>
      <td align="center">0.671</td>
      <td align="center">0.677</td>
      <td align="center">0.377</td>
      <td align="center">0.815</td>
      <td align="center">0.631</td>
      </tr>
      <tr>
      <td align="center">Promedios</td>
      <td align="center">0.820</td>
      <td align="center">0.677</td>
      <td align="center">0.664</td>
      <td align="center">0.380</td>
      <td align="center">0.824</td>
      <td align="center">0.609</td>
      </tr>
      <td align="center">Global</td>
      <td align="center">(NO) </td>
      <td align="center">0.663</td>
      <td align="center">(NO) </td>
      <td align="center">0.393</td>
      <td align="center">(NO) </td>
      <td align="center">0.588</td>
      </tr>
      </tbody>
      </table>    
  
   **Importante**: de acá en más sólamente utilizaremos el score promedio cuando hagamos _K-fold cross-validation_.|

3. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

   <table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td align="center">0.805</td>
   <td align="center">0.659</td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td align="center">0.921</td>
   <td align="center">0.631</td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td align="center">1.0</td>
   <td align="center">0.632</td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td align="center">0.772</td>
   <td align="center">0.661</td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td align="center">0.892</td>
   <td align="center">0.714</td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td align="center">1.0</td>
   <td align="center">0.682</td>
   </tr>
   </tbody></table>


In [46]:
from sklearn.model_selection import ParameterGrid

param_grid = ParameterGrid({"max_depth": [3, 5, None],
                            "criterion": ["gini", "entropy"]})

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for param_comb in param_grid:
    arbol = DecisionTreeClassifier(**param_comb)
    scores = cross_validate(
        arbol,
        X_dev, y_dev,
        cv=skf,
        scoring="accuracy",
        return_train_score=True,
    )
    print(f"(max_depth={param_comb['max_depth']}, criterio={param_comb['criterion']}):")
    print(f"Accuracy (training): {scores['train_score'].mean()}")
    print(f"Accuracy (validación): {scores['test_score'].mean()}")
    print()

(max_depth=3, criterio=gini):
Accuracy (training): 0.8189165360501567
Accuracy (validación): 0.6615822784810126

(max_depth=5, criterio=gini):
Accuracy (training): 0.9279290752351098
Accuracy (validación): 0.6241455696202531

(max_depth=None, criterio=gini):
Accuracy (training): 1.0
Accuracy (validación): 0.6188607594936709

(max_depth=3, criterio=entropy):
Accuracy (training): 0.796351880877743
Accuracy (validación): 0.6643354430379748

(max_depth=5, criterio=entropy):
Accuracy (training): 0.8921904388714733
Accuracy (validación): 0.6489240506329114

(max_depth=None, criterio=entropy):
Accuracy (training): 1.0
Accuracy (validación): 0.6213924050632911



4. ¿Qué conclusiones se pueden sacar de estas tablas?

## Ejercicio 3

### Comparación de algoritmos

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de **encontrar el mejor modelo** de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation.

Hiperparámetros_: Revisar la documentación de cada uno de los hiperparámetros para entender qué posibles hiperparámetros impacten de manera positiva en la construcción del algoritmo.

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

1. Algoritmos a probar:
  - Árboles de decisión. Mínimo 4 hiperparámetros.
  - KNN (k-vecinos más cercanos). Mínimo 3 hiperparámetros.
  - SVM (Support vector machine). Mínimo 2 hiperparámetros.

Detallar los hiperparámetros elegidos para cada algoritmo y explicar la razón del espacio de búsqueda considerado para cada uno de estos, ¿cuántas iteraciones usaron?. A su vez, reportar la performance asociada de aquellos que consideren relevantes (al menos la mejor combinación para cada algoritmo).

In [125]:
from sklearn.model_selection import RandomizedSearchCV

def ejercicio3(modelo, param_grid, X_dev, y_dev, n_iter=10):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

  random_search = RandomizedSearchCV(
      modelo,
      param_distributions=param_grid,
      cv=skf,
      scoring=make_scorer(roc_auc_score,
                          response_method="predict_proba"),
      random_state=0,
      n_iter=n_iter,
      n_jobs=-1)

  random_search.fit(X_dev, y_dev)

  y_pred_proba = random_search.predict_proba(X_eval)[:, 1]

  print(f"Mejores parametros para {modelo}: {random_search.best_params_}")
  print(f"Mejor AUCROC alcanzado en la validación cruzada: {random_search.best_score_}")
  print(f"AUCROC en evaluación: {roc_auc_score(y_eval, y_pred_proba)}")

### Arboles de decisión:

In [ ]:
DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': None,
 'splitter': 'best'}

In [63]:
n_samples_dev = len(X_dev)
n_samples_cv_train = int(n_samples_dev * 0.8)

In [106]:
from scipy.stats import randint, loguniform

In [120]:
dtc_param_grid ={
    "max_depth": [None] + list(range(1, len(X_dev.columns) + 1)),
    "criterion": ["gini", "entropy"],
    "ccp_alpha": loguniform(1e-10, 1),
    "class_weight": [None, "balanced"],
    "min_samples_split": randint(2, int(0.1*n_samples_cv_train)),
    "min_samples_leaf" : randint(1, int(0.1*n_samples_cv_train)),
    'splitter': ['best', 'random']
}

ejercicio3(DecisionTreeClassifier(), dtc_param_grid, X_dev, y_dev, n_iter=500)

Mejores parametros para DecisionTreeClassifier(): {'ccp_alpha': np.float64(3.4341963682884592e-09), 'class_weight': None, 'criterion': 'entropy', 'max_depth': 57, 'min_samples_leaf': 22, 'min_samples_split': 3, 'splitter': 'random'}
Mejor AUCROC alcanzado en la validación cruzada: 0.699855248917749
AUCROC en evaluación: 0.7032258064516129


### KNN (k-vecinos más cercanos):

In [99]:
from sklearn.neighbors import KNeighborsClassifier

KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [127]:
knn_param_grid = {
    "n_neighbors": list(range(1, n_samples_cv_train)),
    "weights": ["uniform", "distance"],
    "p": list(range(1, 11))
}

ejercicio3(KNeighborsClassifier(), knn_param_grid, X_dev, y_dev, 100)

Mejores parametros para KNeighborsClassifier(): {'weights': 'distance', 'p': 1, 'n_neighbors': 44}
Mejor AUCROC alcanzado en la validación cruzada: 0.8644588744588744
AUCROC en evaluación: 0.8170506912442396


### SVM (Support vector machine):

In [130]:
from sklearn.svm import SVC
SVC().get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [144]:
svm_param_grid = [
    {'kernel': ['linear'],  'C': loguniform(1, 1e5), "class_weight": [None, "balanced"]},
    {'kernel': ['poly'],    'C': loguniform(1, 1e5), "class_weight": [None, "balanced"],'degree': list(range(2,11)),   'coef0': loguniform(1e-3, 1e3)},
    {'kernel': ['rbf'],     'C': loguniform(1, 1e5), "class_weight": [None, "balanced"], 'gamma': loguniform(1e-10, 1)},
    {'kernel': ['sigmoid'], 'C': loguniform(1, 1e5), "class_weight": [None, "balanced"], 'gamma': loguniform(1e-5,1e5), 'coef0': loguniform(1e-3, 1e3)}
]
ejercicio3(SVC(probability=True, random_state=0), svm_param_grid, X_dev, y_dev, 100)

Mejores parametros para SVC(probability=True, random_state=0): {'C': np.float64(214.5625214968905), 'class_weight': 'balanced', 'gamma': np.float64(0.0001718307997054956), 'kernel': 'rbf'}
Mejor AUCROC alcanzado en la validación cruzada: 0.9287445887445888
AUCROC en evaluación: 0.9182027649769584


2. Compare los resultados obtenidos en el ejercicio anterior con los siguientes modelos con sus hiperparámetros default.

  - LDA (Linear discriminant analysis)
  - Naïve Bayes

### LDA (Linear discriminant analysis):

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_dev, y_dev)

print(f"AUCROC en entrenamiento: {roc_auc_score(y_dev, lda.predict_proba(X_dev)[:, 1])}")
print(f"AUCROC en evaluación: {roc_auc_score(y_eval, lda.predict_proba(X_eval)[:,1])}")

AUCROC en entrenamiento: 0.9869772998805257
AUCROC en evaluación: 0.7


### Naïve Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_dev, y_dev)

print(f"AUCROC en entrenamiento: {roc_auc_score(y_dev, gnb.predict_proba(X_dev)[:, 1])}")
print(f"AUCROC en evaluación: {roc_auc_score(y_eval, gnb.predict_proba(X_eval)[:, 1])}")

AUCROC en entrenamiento: 0.9680406212664278
AUCROC en evaluación: 0.7105990783410138


¿Qué resultados obtuvo? ¿Qué hiperparámetros podrían ser relevantes explorar en estos modelos? ¿Por qué?

3. ¿Cuál fue el mejor modelo y con qué configuración? Explicar por qué creería que dio mejor (recordando qué hace cada algoritmo y con qué tipo de datos están trabajando).

## Ejercicio 4
### Diagnóstico Sesgo-Varianza.

<span style="color: red;">(no realizar hasta la clase _Sesgo y Varianza_)</span>

En este punto, se pide inspeccionar **tres** de sus mejores modelos encontrados hasta ahora de cada familia de modelos: la mejor configuración para el árbol de decisión y la mejor configuración para SVM. Para ello:

1. Graficar curvas de complejidad para cada modelo, variando la profundidad en el caso de árboles, y el hiperparámetro C en el caso de SVM. Diagnosticar cómo afectan al sesgo y a la varianza esos dos hiperparámetros.
2. Graficar curvas de aprendizaje para cada modelo pero ahora incluya LDA. En base a estas curvas, sacar conclusiones sobre si los algoritmos parecen haber alcanzado su límite, o bien si aumentar la cantidad de datos debería ayudar.
3. Construir un modelo **RandomForest** con 200 árboles. Explorar para qué sirve el hiperparámetro max_features y cómo afecta a la performance del algoritmo mediante una curva de complejidad. Explicar por qué creen que se dieron los resultados obtenidos. Por último, graficar una curva de aprendizaje sobre los parámetros elegidos para determinar si sería útil o no conseguir más datos.


**Atención**: Tener en cuenta que debemos seguir utilizando AUC ROC como métrica para estas curvas.


## Ejercicio 5:
### Evaluación de performance

- La entrega del trabajo estará acompañada de una evaluación en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados.

- Su tarea será estimar la performance (AUCROC) que tendrá su mejor modelo en datos de evaluación (X_held_out).

- Para ello, deberán predecir las **probabilidades** de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUCROC resultante y calcularemos el resultado real. Consideraremos que el **mejor modelo será el que se encuentre más cerca del valor real que calcularemos luego de la entrega**.

- Recomendamos no perder de vista esta evaluación/competencia durante el desarrollo del TP, sobretodo en el momento de separar los datos en los primeros puntos.

- Para que podamos evaluar la performance, junto con la entrega del informe, deberán enviar un archivo con el numero de grupo con dos digitos en formato csv con la columna `output` y el valor obtenido con 4 decimales (se subirá un ejemplo cuando se publiquen los datos de la competencia) y un valor esperado de AUCROC: `GG_y_pred_held_out_AUCROC`.

    - Ej.: el grupo tres cree que obtuvo un valor de 0.7321 de AUCROC deberá submitear un archivo llamado: `03_y_pred_held_out_7321.csv`.

- Los datos podrán encontrarlos en este [link](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

- Las decisiones de este punto pueden desarrollarse hasta en una carilla, aunque con media debería alcanzar.


## Ejercicio 6:
### Conclusiones

Escribir como mínimo en un párrafo, una conclusión del trabajo realizado, incluyendo problemas encontrados y
aspectos no incluidos en el enunciado que hayan sido abordadas durante el desarrollo.

---
## Entregables
- Contarán con un esqueleto en formato Jupyter Notebook en donde podrán intercalar celdas para reportar y responder a los ítems de cada ejercicio.
- Los entregrables serán
    - Un informe en formato .pdf (**digital**) que responda a los ítems de este enunciado respetando la cantidad de espacio máximo por cada ítem. Nombrarlo siguiendo el formato `GG_Nombre_de_grupo`
    - Adjuntar el notebook final en formatos .pdf e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (verificarlo haceindo: Kernel -> Restart and Run All).
    - Las predicciones del *held out* del punto 5 en formato csv.
- Habŕa una entrega intermedia obligatoria que deberán hacer antes del 17 de abril de 2025 a las 17:00hs. Para esta entrega deberán enviar el código que resuelve los primeros 3 ejercicios.
- La **fecha** y **hora límite** de entrega está determinada en el campus de la materia.
- El trabajo deberá elaborarse en grupos de 5 personas.
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.


### Importante: sobre el uso de ChatGPT y grandes modelos de lenguaje

En este trabajo no estará explícitamente prohibido pero si fuertemente desaconsejado, consideramos a este trabajo práctico una importante herramienta de aprendizaje donde el uso de GPT puede ser perjudicial. En caso de usarlo se pide aclararlo en el informe y especificar cómo y en donde se utilizó. Así como expresar su opinión sobre la respuesta generada por el modelo pudiendo estar a favor o en contra de lo propuesto por este. Pueden adjuntar el link a la conversación con el modelo.

**Nota**: Agradecemos a [Martín García Sola](https://ar.linkedin.com/in/martin-e-garcia-sola) por la asistencia biológica en la confección de este Trabajo Práctico.